# Rapport de projet numérique : Modélisation de la sortie d'une foule d'une salle

## Introduction

Le but de ce projet numérique est de modéliser la sortie de plusieurs personnes se trouvant dans une salle.

### Importation des librairies

In [1]:
import itertools
import random
import numpy as np
import matplotlib.animation as anim
import matplotlib.pyplot as plt
import os

### Variables d'entrées

Un nombre n de personnes seront placé aléatoirement dans une salle de taille x*y. On veillera à ce que deux personnes ne se superposent pas. Une personne sera représenté par une classe dans laquelle seront stocké sa position, sa vitesse, son rayon, la vitesse maximum qu'il peut atteindre et son numéro.

In [2]:
def initial(n,x,y,vm=7,r=0.3):
    """
    Placement aléatoire de n personnes et affectation de leurs caractéristiques
    """
    PosVi_tab=rd.rand(n,2,2)
    PosVi_tab[:,0,0]=PosVi_tab[:,0,0]*x+1
    PosVi_tab[:,0,1]=PosVi_tab[:,0,1]*y+1
    classe_tab=[]
    R=[]
    
    for num_part , PosVi in enumerate(PosVi_tab):
        
        ri=np.absolute(np.random.normal(r,0.05))
        v_max_i=np.absolute(np.random.normal(vm,0.8))
        R.append(ri)
        classe_tab=classe_tab+[personne(PosVi[0,0],PosVi[0,1],PosVi[1,0],PosVi[1,1],ri,v_max_i,num_part)]
    
    classe_tab=np.array(classe_tab)
    #Replacement des personnes qui se superposent
    ls = np.arange(0,len(classe_tab))
    couple = list(itertools.combinations(ls,2))
    a=1 ; p=0
    
    while a!=0 and p<=100:
        
        a=0
        
        for ij in couple:
            
            i = ij[0]
            j = ij[1]
            #Test de superposition
            if classe_tab[i].superpose(classe_tab[j]):
                
                a=a+1
                classe_tab[i].x = rd.rand()*x+1
                classe_tab[i].y = rd.rand()*y+1
        p=p+1
    
    if p==101 :
        
        input('Trop de personnes dans la salle')
    for num_pers , pers in enumerate(classe_tab):
        
        PosVi_tab[num_pers,0,0]=pers.x
        PosVi_tab[num_pers,0,1]=pers.y

    return PosVi_tab , classe_tab , R

## Les interactions de personne à personne

### Définition de la classe personne

### Premier modèle d'intéraction personne à personne

La première façon de modéliser les intéractions entre personnes est de considérer des forces entre les particules. Dans ce cas les personnes sont modéliser comme des particules subissant des intéractions.